### NOTA: NON ELIMINARE GLI OUTPUT PER VEDERE I RISULTATI

## Caricamento Dataset

In [ ]:
import pandas as pd
import csv
import sys

CSV_PATH = "TREC-05.csv"  # percorso CSV
csv.field_size_limit(sys.maxsize)

df_trec = pd.read_csv(
    CSV_PATH,
    engine='python',
)

CSV_PATH = "Nazario.csv"  # percorso CSV
csv.field_size_limit(sys.maxsize)

df_nazario = pd.read_csv(
    CSV_PATH,
    engine='python',
)

CSV_PATH = "Nigerian_Fraud.csv"  # percorso CSV
csv.field_size_limit(sys.maxsize)

df_nigeriane = pd.read_csv(
    CSV_PATH,
    engine='python',
)

CSV_PATH = "TREC-07.csv"  # percorso CSV
csv.field_size_limit(sys.maxsize)

df_trec07 = pd.read_csv(
    CSV_PATH,
    engine='python',
)

In [ ]:
df_trec07_1 = df_trec07[df_trec07["label"]==1]  #Dopo vari tentativi è stata notata l'esigenza di ulteriori entry con
                                                #label 1 (Phishing)

Concatenazione Dataset

In [ ]:
df = pd.concat([df_trec, df_nazario, df_nigeriane,df_trec07_1], ignore_index=True)

Funzione per visualizzare lo stato del dataset

In [ ]:
def dataset_metrics(df, name="Dataset", label_col="label"):
    print(f"\n📊 Metriche - {name}")
    print("-" * 50)
    
    print(f"Numero righe: {df.shape[0]}")
    print(f"Numero colonne: {df.shape[1]}")
    
    print("\nColonne:")
    print(df.columns.tolist())
    
    print("\nValori mancanti per colonna:")
    print(df.isnull().sum())

    # Conteggio label 0 e 1
    if label_col in df.columns:
        print(f"\nDistribuzione label ({label_col}):")
        print(df[label_col].value_counts().sort_index())
    else:
        print(f"\n⚠️ Colonna label '{label_col}' non trovata nel DataFrame")


In [ ]:
dataset_metrics(df_trec07_1)

In [ ]:
dataset_metrics(df)

In [ ]:
dataset_metrics(df_trec)

In [ ]:
dataset_metrics(df_trec07)

In [ ]:
dataset_metrics(df_nigeriane)

In [ ]:
dataset_metrics(df_nazario)

Analisi Indici con valori mancanti sui campi: "receiver"

In [ ]:
import pandas as pd

def trova_missing_df(df, campo):
    """
    Restituisce le righe di un DataFrame in cui il campo specificato è mancante (NaN o None)
    """
    # Seleziona le righe dove il campo è NaN o None
    missing_rows = df[df[campo].isna()]
    return missing_rows

# Esempio di utilizzo
campo_da_controllare = "receiver"
mancanti = trova_missing_df(df, campo_da_controllare)

print(f"Istanze con '{campo_da_controllare}' mancante:")
print(mancanti)

In [ ]:
df.iloc[176]        #visualizzo un istanza mancante per vedere se veramente manca

Eliminazione delle righe con un campo mancante ("sender", "receiver", "urls", "label", "date") 

In [ ]:
def drop_rows_with_nan_in_columns(df, columns):
    """
    Rimuove le righe che hanno NaN in una qualunque delle colonne specificate.
    """
    cols = [c for c in columns if c in df.columns]

    print(f"Colonne controllate: {cols}")

    initial_rows = len(df)
    df_clean = df.dropna(subset=cols)           #eliminazione NaN
    removed_rows = initial_rows - len(df_clean)

    print(f"Righe rimosse: {removed_rows}")

    return df_clean


cols_to_drop = ["sender","receiver","date","label","urls"]  #campi da controllare -> mancano subject e body perchè sono casi interessanti per la detection

df = drop_rows_with_nan_in_columns(df,cols_to_drop)

In [ ]:
dataset_metrics(df)         #verifico che solo body e subject possono avere valori non NaN

## Pulizia Caratteri Strani e Punteggiatura

Funzione che restituisce True se vi sono troppi caratteri strani nel testo, o troppa punteggiatura. 

La rimozione avviene verificando una percentuale, al superamento di una soglia deve indicare la volontà di eliminare la riga

In [ ]:
import string
import unicodedata

def is_standard_char(c):
    """
    Restituisce True se il carattere è 'standard' (lettere e numeri ASCII)
    """
    return c in string.ascii_letters + string.digits + ' '

def is_punctuation(c):
    """
    Restituisce True se il carattere è punteggiatura comune
    """
    return c in string.punctuation

def is_weird_char(c):
    """
    Restituisce True se il carattere è 'strano', cioè non standard e non punteggiatura
    """
    return not is_standard_char(c) and not is_punctuation(c)

def should_discard(text, weird_threshold=0.5, punctuation_threshold=0.5):
    """
    Decide se scartare un testo basandosi sulla percentuale di caratteri strani o punteggiatura
    """
    if pd.isna(text):
        return False

    if not text.strip():  # se il testo è vuoto o solo spazi
        return False

    total_chars = len(text)
    if total_chars == 0:
        return False

    weird_count = sum(1 for c in text if is_weird_char(c))          #Somma dei caratteri strani
    punctuation_count = sum(1 for c in text if is_punctuation(c))   #Somma dei caratteri usati come punteggiatura

    weird_ratio = weird_count / total_chars                         #Percentuale di caratteri strani sul corpo totale
    punctuation_ratio = punctuation_count / total_chars             #Percentuale di caratteri usati come punteggiatura sul corpo totale

    # scarta se supera le soglie
    if weird_ratio > weird_threshold:
        return True
    if punctuation_ratio > punctuation_threshold:
        return True

    return False

In [ ]:
dataset_metrics(df)

In [ ]:
def clean_text_dataset(df,  text_column = "body", weird_threshold=0.4, punctuation_threshold=0.5):

    # Applicare il filtro
    discard_mask = df[text_column].apply(
        lambda x: should_discard(x, weird_threshold, punctuation_threshold)     #Verifica gli indici da eliminare
    )
    discarded_count = discard_mask.sum()                                        #somma delle entry eliminate

        # Indici dei testi scartati
    discarded_indices = df.index[discard_mask].tolist()                         #lista degli indici eliminati

    df_clean = df[~discard_mask].copy().reset_index(drop=True)                  #Eliminazione
    return df_clean, discarded_count, discarded_indices         #returns

In [ ]:
df_check = df                                           #Usato successivamente per debugging
dataset_metrics(df)
df, count, lista_strani = clean_text_dataset(df)        #pulizia di df
print(count)                                            #Print del numero di entry eliminate
dataset_metrics(df)                                     #Stampa, utile per vedere il numero attuale di entry nel dataset

In [ ]:
def stampa_body(dataset):
    """
    Itera su tutte le righe di un DataFrame e stampa solo il campo 'body'.
    """
    for index, row in dataset.iterrows():
        print(f"Riga {index}: {row['body']}")
        print("-" * 40)

In [ ]:
df_cleaned = df_check.loc[lista_strani].copy().reset_index(drop=True)       #df_cleaned contiene solo le entry eliminate
dataset_metrics(df_cleaned) #Controllo il numero di entry eliminate (deve coincidere con il count restituito precedentemente)
stampa_body(df_cleaned)     #Controllo delle entry eliminate

Pulizia dimensione del dataset (TOKEN)

In [ ]:
dataset_metrics(df)

In [ ]:
def build_prompt(metadata_list, body):          #Unisce le informazioni relative a "sender" "receiver" "date" "subject"
                                                #al body. Per distinguere i campi sono stati usati dei separatori | .
                                                #Infine si aggiunge anche l'istruzione usata per la fase di detection.
                                                #build_prompt restituisce in uscita la struttura del prompt usato anche
                                                #in fase di detection.

    #Formattazione delle informazioni aggiuntive
    #FIELDS = ['sender', 'receiver', 'date', 'subject']
    metadata_prompt = (
        "sender: " + str(metadata_list[0]) + " | " +
        "receiver: " + str(metadata_list[1]) + " | " +
        "date: " + str(metadata_list[2]) + " | " +
        "subject: " + str(metadata_list[3])
    )
    
    #Aggiungo il body
    body_text = metadata_prompt + " | body: " + body

    #Aggiungo l'istruzione
    prompt = f"""### Instruction:
You are a classification model specializing in emails, and your job is to detect phishing: respond only with \"0\" if it is not phishing or \"1\" if it is phishing, without explanations, symbols, additional letters, or other characters.
### Input:
{body_text}
### Response:
"""

    return prompt

In [ ]:
def clean_max_token(df):


    #Caricamento del tokenizer, per poter trasformare il corpo del messaggio di ogni entry in token
    from transformers import AutoTokenizer

    LORA_ADAPTER_PATH = "./llama-finetuned-final"

    tokenizer = AutoTokenizer.from_pretrained(LORA_ADAPTER_PATH)    #caricamento del tokenizer

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    MAX_TOKENS = 1024           #NUMERO MASSIMO DI TOKEN

    #Lista delle istanze valide
    filtered_dataset = []   

    #Lista delle istanze non valide (il cui prompt generato supera i 1024 token)    
    overflow_instances = []

    for idx, row in df.iterrows():

        #Lista avente le informazioni aggiuntive da inserire nel body, in un ordine prestabilito
        metadata_list = [
            row["sender"],
            row["receiver"],
            row["date"],
            row["subject"]
        ]

        #Verifico che subject sia NaN.
        if pd.isna(row["subject"]):
            metadata_list[3]=""     #Sostituzione NaN con carattere vuoto

        body = row["body"]          

        #Verifico che body sia NaN
        if pd.isna(body):
            body=""                 #Sostituzione NaN con un body vuoto    

        prompt = build_prompt(metadata_list, body)  #Costruzione prompt

        #Trasformazione in token
        tokens = tokenizer(
            prompt,
            add_special_tokens=True,
            return_tensors="pt",
            truncation=False
        )

        num_tokens = tokens["input_ids"].shape[1]   

        #Smistamento delle righe nelle rispettive liste
        if num_tokens <= MAX_TOKENS:
            filtered_dataset.append(row)
        else:
            overflow_instances.append(row)

        # Stampa progresso ogni 5000 iterazioni
        if (idx + 1) % 5000 == 0:
            print(f"Elaborate {idx + 1} righe su {len(df)}...")

    print(f"Istanze totali: {len(df)}")
    print(f"Istanze valide (<=1024): {len(filtered_dataset)}")
    print(f"Istanze scartate: {len(overflow_instances)}")

    #Restituisco le liste trasformate in Dataset
    return pd.DataFrame(filtered_dataset).reset_index(drop=True), pd.DataFrame(overflow_instances).reset_index(drop=True)

In [ ]:
df, df_sporco = clean_max_token(df)

In [ ]:
dataset_metrics(df)

In [ ]:
dataset_metrics(df_sporco)

Scrittura in un File per visualizzare le righe eliminate a causa della lunghezza eccessiva

In [ ]:
with open("output.txt", "w", encoding="utf-8") as f:
    for i in range(0, 8):
        f.write(f"Riga: {i}\n")
        f.write(df_sporco.iloc[i]["body"] + "\n\n")  

Installazione di sklearn (Da eseguire almeno la prima volta)

In [ ]:
!pip install scikit-learn


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- FUNZIONE DI PULIZIA ---
def rimuovi_duplicati_smart(df,SOGLIA_SIMILARITA):

    initial_len = len(df)

    # --- FASE 1: DUPLICATI ESATTI (100%) ---
    # Identifichiamo quali sono i duplicati PRIMA di cancellarli per poterli stampare
    #Complessità O(n) per fare un filtraggio iniziale

    mask_duplicati_esatti = df.duplicated(subset='body', keep='first')
    df_esatti_eliminati = df[mask_duplicati_esatti].copy()
    
    num_esatti = len(df_esatti_eliminati)
    print(f"   -> Trovati {num_esatti} duplicati ESATTI (100%).")

    # STAMPA ESATTI (Max 50)
    if num_esatti > 0:
        print(f"   🔻 ESEMPI RIMOSSI (Esatti) - Mostro i primi {min(50, num_esatti)}:")
        for i, text in enumerate(df_esatti_eliminati['body'].head(50)):
            text_str = str(text).replace('\n', ' ')
            print(f"      [X] {text_str[:100]}...") # Stampa i primi 100 caratteri
        print("-" * 50)
    

    # Rimozione effettiva Esatti
    df = df[~mask_duplicati_esatti].reset_index(drop=True)
    len_after_exact = len(df)

    # Se il dataset è diventato troppo piccolo, ci fermiamo in quanto non è possibile eseguire la funzione su meno di 2 righe
    if len_after_exact < 2:
        return df

    # --- FASE 2: QUASI DUPLICATI (80%) ---
    #Complessità di O(n^2) su un numero di campioni ridotti grazie al filtraggio di quelli identici
    print("   -> Calcolo similarità (quasi-duplicati)...")
    
    corpus = df['body'].fillna("").astype(str).tolist()

    vectorizer = TfidfVectorizer(stop_words='english', min_df=1)        

    tfidf_matrix = vectorizer.fit_transform(corpus)  

    cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)       #Applicazione della cosine similarity
    upper_tri = np.triu(cosine_sim_matrix, k=1)             #Eliminazione dei controlli duplicati
    
    to_drop = set()                                         #to_drop indica le entry da eliminare
    rows, cols = np.where(upper_tri > SOGLIA_SIMILARITA)    #Colonne che superano la similarità soglia

    #Aggiunta delle entry da eliminare
    for r, c in zip(rows, cols):
        if r not in to_drop:
            to_drop.add(c)
            
            #Stampa delle entry che hanno fornito una similarità maggiore della soglia
            print(f"      [~] Indici ({r}, {c}) - Similarità: {cosine_sim_matrix[r,c]:.2f}")
            print(f"         -> {df.iloc[r]['body'][:100]}...")         
            print(f"         -> {df.iloc[c]['body'][:100]}...\n")

    num_quasi = len(to_drop)        #Numero di entry da eliminare che superano la soglia fissata
    
    # STAMPA QUASI DUPLICATI (Max 50)
    if num_quasi > 0:
        '''
        print(f"   -> Trovati {num_quasi} messaggi QUASI identici (>80%).")
        print(f"   🔻 ESEMPI RIMOSSI (Simili) - Mostro i primi {min(50, num_quasi)}:")
        
        # Recuperiamo le righe da eliminare usando gli indici
        df_quasi_eliminati = df.iloc[list(to_drop)]
        
        for i, text in enumerate(df_quasi_eliminati['body'].head(50)):
            text_str = str(text).replace('\n', ' ')
            #print(f"      [~] {text_str[:100]}...")
        print("-" * 50)
        '''
        # Rimozione effettiva dei Quasi Duplicati
        df_clean = df.drop(index=list(to_drop)).reset_index(drop=True)
    else:
        print("   -> Nessun quasi duplicato trovato.")
        df_clean = df.reset_index(drop=True)

    totale_rimossi = initial_len - len(df_clean)
    print(f"   Dataset Finale: {len(df_clean)} campioni (Totale rimossi: {totale_rimossi})")
    
    return df_clean

In [ ]:
#stampa sul file csv per poter controllare manualmente i duplicati
df.to_csv("df.csv", sep=",", encoding='utf-8', index=False)

In [ ]:
dataset_metrics(df)

df = rimuovi_duplicati_smart(df,0.75)           #Eliminazione delle righe simili con una soglia di 0.75

dataset_metrics(df)


Memorizzazione del Dataset dopo la pulizia (Opzionale)

In [ ]:
df.to_json("dopo_duplicati.json", orient="records", force_ascii=False, indent=4)

Caricamento del Dataset dopo la pulizia dei duplicati (Opzionale)

In [ ]:
import pandas as pd
import csv
import sys

df = pd.read_json("dopo_duplicati.json")

Installazione libreria per se i testi sono costituiti da parole reali nel dizionario inglese

In [ ]:
!pip install wordfreq


In [ ]:
from wordfreq import zipf_frequency

def english_word_ratio(text, threshold=2.5):

    if pd.isna(text):
        return 0        #se il testo è NaN 
    words = text.split()
    if not words:
        return 0        #se il testo è vuoto

    valid = [w for w in words if zipf_frequency(w, "en") > threshold]   #Ogni parola è associata a un valore strettamente
                                                                        #positivo (>=0). Una parola rara ma reale assume
                                                                        #valori maggiori di 2 (>=2). Parole non inglesi assumono
                                                                        #Valori fra 0 e 2 (0 <= x <= 2). Tanto più le parole
                                                                        #sono comuni maggiore sarà il valore ad esse 
                                                                        #associato.
                                                                        #threshold indica che vogliamo eliminare solo le 
                                                                        #parole con valore minore di 2.5
    
    #Rapporto tra parole valide e parole totali
    return len(valid) / len(words)

In [ ]:
def count_entries_above_ratio(df, ratio_threshold, text_column="body" , wordfreq_threshold=2.5):

    count = 0

    for text in df[text_column]:

        #Ratio del testo considerato
        ratio = english_word_ratio(text, threshold=wordfreq_threshold)  

        if ratio > ratio_threshold:
            count += 1      #Conteggio delle entry valide

    return count


In [ ]:
print(len(df))
print(count_entries_above_ratio(df,0.8))    #I testi validi ammontano a 42624

In [ ]:
def filter_entries_by_ratio(df, ratio_threshold, text_column="body", wordfreq_threshold=2.5):
    # Calcolo il ratio per ogni riga
    ratios = df[text_column].apply(lambda x: english_word_ratio(x, threshold=wordfreq_threshold))

    # Filtra il DataFrame usando la soglia
    df_dest = df[ratios > ratio_threshold].copy()
    
    #Dataset sorgente filtrato
    return df_dest


In [ ]:
df_prova = filter_entries_by_ratio(df,0.8)
dataset_metrics(df_prova)   #Controllo per verificare che il numero di righe coincida con 42624 righe


FINE PULIZIA DATASET COMPLESSIVO.
Split in 2 dataset -> Email Normali -> Email di Phishing

In [ ]:
df0 = df[df["label"] == 0]  #df0 -> DataFrame NON di Phishing
df1 = df[df["label"] == 1]  #df1 -> DataFrame di Phishing

In [ ]:
dataset_metrics(df0)

In [ ]:
dataset_metrics(df1)

## Verifica delle email di risposta

Questa funzione permette di verificare quante email contengono il prefisso **"Original Message"**.

Il prefisso *"Original Message"* è tipicamente presente nelle email che rispondono ad altri messaggi; la sua individuazione consente quindi di confermare con certezza la presenza di email di risposta all’interno del dataset.


In [ ]:
def has_original_message(text, pattern="Original Message"):
    """
    Restituisce True se il testo contiene 'Original Message' (case-insensitive)
    """
    if text is None:
        return False

    return pattern.lower() in str(text).lower()

In [ ]:
def count_pattern(df, body_col="body", pattern="Original Message"):
    """
    Stampa il numero di righe in cui il campo body contiene 'Original Message'
    """
    if body_col not in df.columns:
        print(f"⚠️ Colonna '{body_col}' non trovata nel DataFrame")
        return 0

    #Restituzione di una maschera che indica quali righe presentano "Original Message"
    mask = df[body_col].astype(str).str.contains(pattern, case=False, na=False)
    count = mask.sum()

    print(f"Istanze con '{pattern}' nel {body_col}: {count}")

    return count

## Verifica delle email che presentano dei file al loro interno.

Questa funzione verifica che i file trovati non siano interni a dei link. In entrambi i casi sono stati usati dei regex per l'individuazione di entrambi, per poi verificare che la posizione dei file non coincidesse con quella dei link. 

I file sono trovati tramite l'utilizzo di un estensione.

In [ ]:
import re

def contiene_file_non_link(testo: str) -> bool:

    #Estensioni famose
    FILE_EXTENSIONS = [
    # Documenti
    "pdf", "doc", "docx", "csv", "xls", "xlsx",
    "txt", "ppt", "pptx", "odt", "ods", "odp", "rtf",
    
    # Archivi compressi
    "zip", "rar", "7z", "tar", "gz", "bz2",
    
    # Immagini
    "jpg", "jpeg", "png", "gif", "bmp", "tiff", "svg", "webp",
    
    # Audio
    "mp3", "wav", "ogg", "flac", "aac", "m4a",
    
    # Video
    "mp4", "avi", "mov", "mkv", "flv", "wmv", "webm"
]

    if not testo:
        return False    #Verifica che il testo non sia vuoto

    if testo is None or pd.isna(testo):
        return False    #Verifica che il testo non sia NaN

    testo_lower = testo.lower()

    #All'interno del dataset dei link risultavano separati da degli spazi, tramite le prossime righe di codice
    #li uniamo per fornirgli una forma corretta.
        # Esempio: "h t t p : / / example.com" -> "http://example.com"
    testo_lower = re.sub(r"h\s*t\s*t\s*p\s*s?\s*:", "http:", testo_lower)
    testo_lower = re.sub(r"w\s*w\s*w\s*\.", "www.", testo_lower)

    # Trova tutti i link
    link_regex = re.compile(r"https?://\S+|www\.\S+")
    
    #Ottenimento della posizione dei link nel testo fornito
    link_spans = [m.span() for m in link_regex.finditer(testo_lower)]

    # Trova tutte le estensioni di file
    file_regex = re.compile(
        rf"\b\S+\.({'|'.join(FILE_EXTENSIONS)})\b"
    )

    #Si itera per ogni file trovato nel testo
    for file_match in file_regex.finditer(testo_lower):
        
        #si separa l'inizio e la fine del file
        file_start, file_end = file_match.span()

        #Verifica se il file cade dentro un link, iterando fra tutti i link trovati
        is_inside_link = any(
            link_start <= file_start and file_end <= link_end
            for link_start, link_end in link_spans
        )

        #Se NON è dentro un link → file valido
        if not is_inside_link:
            return True

    return False

In [ ]:
#Conteggio delle entry con un file nel corpo del messaggio
def conta_file(df,col_name="body"):
    count = 0

    for i in range(len(df)):
        testo = df.iloc[i][col_name]
        
        if contiene_file_non_link(testo):
            count += 1

    return count

## Verifica delle email con numeri di telefono, email o riferimenti di contatto

In [ ]:
import pandas as pd
import re

def has_phone_contact(text):
    if pd.isna(text): 
        return False
    
    # Convertiamo in stringa e minuscolo
    text_str = str(text).lower()
    
    #Analizzando il codice sono state notate delle forme diverse per indicare il numero di telefono
    #Dopo un analizzazione del dataset sono state notate le seguenti forme
    patterns_list = [
        # --- PATTERN ORIGINALI ---
        r"\b\d{3}-\d{3}-\d{3}\b",           # xxx-xxx-xxx (9 cifre)
        r"\b\d{3}-\d{3}-\d{4}\b",           # xxx-xxx-xxxx (Standard trattini)
        r"\b\d{3}\s\d{3}\s\d{4}\b",         # xxx xxx xxxx (Spazi)
        r"\b\d-\d{3}-\d{3}-\d{4}\b",        # x-xxx-xxx-xxxx (International)
        r"\b\d\.\d{3}\.\d{3}\.\d{4}\b",     # x.xxx.xxx.xxxx (11 cifre punti)
        
        r"\b\d{10}\b",                      # 1) xxxxxxxxxx (10 cifre unite, senza spazi)
        r"\(\d{3}\)\s\d{3}-\d{4}\b",        # 2) (xxx) xxx-xxxx (Parentesi e spazio)
        r"\b\d{3}\s\d{3}-\d{4}\b",          # 3) xxx xxx-xxxx (Spazio misto trattino)
        r"\b\d{3}/\d{3}-\d{4}\b",           # 4) xxx/xxx-xxxx (Slash misto trattino)
        r"\b\d{3}\.\d{3}\.\d{4}\b",         # 5) xxx.xxx.xxxx (10 cifre con punti)
        r"\(\d{3}\)\d{3}-\d{4}\b"           # 6) (xxx)xxx-xxxx (Parentesi senza spazio)
    ]
    
    # Uniamo tutti i pattern con l'operatore OR "|"
    combined_phone_regex = "|".join(patterns_list)
    
    #Per rendersi conto dei numeri di telefono sono state usate anche delle parole di riferimento.
    #Analizzando il corpo del dataset sono state notate una serie di parole chiave.
    keywords = [
        
        "phone", "cell", "tel", "mobile", "call", "fax", "ph", "mob", "Telephone",
        "hotline", "helpline", "toll free", "toll-free", "dial", "direct line",
        "sms", "text message", "whatsapp"
    ]
    
    # Creiamo Regex per parole intere
    pattern_keywords = r"\b(" + "|".join(keywords) + r")\b"
    
    # --- CONTROLLO FINALE ---
    # Cerca se esiste ALMENO uno dei formati numerici
    found_strict = bool(re.search(combined_phone_regex, text_str))
    
    # Cerca se esiste ALMENO una keyword
    found_keyword = bool(re.search(pattern_keywords, text_str, flags=re.IGNORECASE))
    
    return found_strict or found_keyword        #se trova una keyword o almeno un numero di telefono restituisce True

In [ ]:
def has_email_contact(text):

    if pd.isna(text):
        return False

    text_str = str(text).lower()

    regex_email = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'     #formato email
    emails_found = re.findall(regex_email, text_str, flags=re.IGNORECASE)   #Ricerca di email nel body

    return len(emails_found) > 0    #Se nel corpo vi è almeno una email trovata restituisce True

In [ ]:
import pandas as pd
import re

def check_information_content(text):

    if pd.isna(text):
        return False
    text_str = str(text)
    
    # Controllo Telefono
    has_phone = has_phone_contact(text_str)
        
    # Controllo Email
    has_email = has_email_contact(text_str)
    
    # In modo analogo con quanto visto con il controllo dei numeri di telefono, anche in questo caso usiamo delle
    #parole chiave.
    general_contact_keywords = [

        "contact us", "contact me", "contact our",         
        "call us", "call me", "call our",                   
        "email us", "email me", "write to", "write us",     
        "reply to", "reply via", "respond to",           
        "reach us at", "reach me at",                      
        "get in touch", "feel free to call",                
        
        "private email", "personal email", "direct email",  
        "private number", "private line", "direct line",    
        "confidential email", "confidential number",       
        "contact the bank", "contact the barrister",        
        "contact the secretary", "contact the agent",       
        "send a fax", "send an email",                      
        "get back to me", "return this email",              
        
        "contact support", "contact security",              
        "call customer service", "call support",            
        "contact the resolution center",                   
        "verify by phone", "verify by email",               
        "fax verification",                                
        "assistance line", "help line",                     
       
        "call now", "call for details",                     
        "order by phone", "order via email",               
        "unsubscribe via email", "remove via email",       
        "call to remove", "reply to unsubscribe",           
        "questions? call", "questions? email"               
    ]
    
    # Creiamo il pattern per cercare le frasi intere
    pattern_general = r"\b(" + "|".join(general_contact_keywords) + r")\b"
    
    # Verifichiamo se c'è la keyword
    has_general_kw = bool(re.search(pattern_general, text_str, flags=re.IGNORECASE))
    

    # (Ha il telefono E la keyword) OPPURE (Ha la keyword E l'email)
    # Normalmente una mail nel caso in cui fornisce delle opzioni di contatto è strutturata in modo da avere
    # almeno un numero di telefono o un email e delle richieste di contatto che vengono identificate tramite
    # parole chiave, ad esempio "contact us" o "email us"
    return (has_phone and has_general_kw) or (has_general_kw and has_email)

In [ ]:
#Funzione per il conteggio di 3 casi separati
# -> Solo delle mail sono presenti nel corpo
# -> Solo dei numeri di telefono sono presenti nel corpo
# -> Vi è la presenza di opzioni di contatto
def analizza_dataset(df,col_name="body"):
   
    count_only_email = 0    # Ha l'email MA NON il telefono
    count_only_phone = 0    # Ha il telefono MA NON l'email
    count_both = 0          # Ha ENTRAMBI
    
    total_samples = len(df)

    for text in df[col_name]:

        if pd.isna(text):
            continue
            
        text_str = str(text).lower()

        found_phone = has_phone_contact(text_str)

        found_email = has_email_contact(text_str)

        if check_information_content(text_str):     #Se vi sono opzioni di contatto
            count_both += 1
        elif found_phone and not found_email:       #Se vi sono solo numeri di telefono
            count_only_phone += 1
        elif found_email and not found_phone:       #Se vi sono email
            count_only_email += 1

    # --- STAMPA RISULTATI ---
    print(f"Totale campioni: {total_samples}")
    print("-" * 30)
    print(f"1. SOLO Email (no telefono): {count_only_email}")
    print(f"2. SOLO Telefono (no email): {count_only_phone}")
    print(f"3. Opzioni di Contatto (Email o Telefono): {count_both}")
    print("-" * 30)


## Istogramma

In [ ]:
from collections import Counter
import math

def istogramma(df, n_bins,col_name = "body"):

    # lunghezze del testo
    lengths = df[col_name].astype(str).apply(len)

    min_len = lengths.min() 
    max_len = lengths.max()

    #Casi limite
    if min_len == max_len:
        print("Tutti i testi hanno la stessa lunghezza:", min_len)
        return

    #Dimensione dei bucket automatica
    bin_size = math.ceil((max_len - min_len + 1) / n_bins)

    # assegnazione ai bucket
    bins = ((lengths - min_len) // bin_size) * bin_size + min_len

    # conteggio
    histogram = Counter(bins)

    # stampa dei conteggio per ogni bucket
    print(f"Istogramma lunghezze (caratteri) — {n_bins} bucket\n")
    for b in range(min_len, max_len + 1, bin_size):
        count = histogram.get(b, 0)
        bar = "#"
        print(f"{b:4d}–{b+bin_size-1:4d} | {bar} ({count})")


## Funzione per eliminare i testi troppo lunghi

In [ ]:
def elimina_lunghe(df, max_chars, colonna="body"):

    # assicuriamoci che siano stringhe
    df[colonna] = df[colonna].astype(str)
    
    # eliminazione delle righe troppo lunghe
    df_filtrato = df[df[colonna].str.len() <= max_chars].copy()
    
    return df_filtrato


## Funzione per analisi del tono e contesto

Questa funzione permette di verificare dei pattern comuni nelle mail di phishing, come il cercare di mettere fretta

In [ ]:
#Parole chiave nelle mail di phishing

phishing_urgency_keywords = [

    "urgent",
    "immediately",
    "as soon as possible",
    "asap",
    "now",
    "today",
    "within 24 hours",
    "within 48 hours",
    "right away",
    "final notice",
    "last chance",
    "time-sensitive",
    "expires",
    "deadline",
    "act fast",
    "limited time",

    "suspended",
    "terminated",
    "blocked",
    "disabled",
    "locked",
    "restricted",
    "closed",
    "cancelled",
    "deactivated",
    "compromised",
    "unauthorized activity",
    "security breach",
    "risk",
    "violation",

    "verify your account",
    "confirm your identity",
    "update your information",
    "reset your password",
    "click",
    "open the attachment",
    "respond immediately",
    "take action",
    "sign in to avoid",

    "security team",
    "support team",
    "it department",
    "administrator",
    "compliance",
    "customer service",
    "official notice",
    "automated message",

    "important",
    "attention",
    "warning",
    "alert",
    "critical",
    "serious",
    "unusual activity",
    "suspicious login",

    "your account has been",
    "action required",
    "failure to comply",
    "if you do not",
    "to avoid suspension",
    "we noticed unusual",
    "for your security"
]


In [ ]:
def count_keywords(df, body_col="body", keywords=None):

    if body_col not in df.columns:      #verifica che la colonna vi sia (successivamente il corpo non si chiamerà più body)
        print(f" Colonna '{body_col}' non trovata nel DataFrame")
        return 0

    #regex
    escaped_keywords = [re.escape(k) for k in keywords]
    pattern = "|".join(escaped_keywords)

    #ricerca delle entry che presentano almeno una parola chiave fra quelle indicate
    mask = (
        df[body_col]
        .astype(str)
        .str.contains(pattern, case=False, na=False, regex=True)
    )

    count = mask.sum()      #il numero di righe che presentano le parole chiave indicate

    print(f"Istanze con almeno una keyword nel {body_col}: {count}")

    return count

## Conteggio dei link presenti nelle mail

In [ ]:
from collections import Counter

def conta_url(df, colonna="urls"):

    counts = Counter(df[colonna])
    print("Conteggio righe per numero di URL:")
    for k in sorted(counts):
        print(f"{k} URL : {counts[k]} righe")

Analisi Euristiche

In [ ]:
analizza_dataset(df0)
analizza_dataset(df1)

In [ ]:
conta_url(df0)
conta_url(df1)

In [ ]:
print(conta_file(df0))
print(conta_file(df1))

In [ ]:
count_pattern(df0)
count_pattern(df1)

In [ ]:
dataset_metrics(df)

## Unione delle informazioni aggiuntive nel body



In [ ]:
def compatta_detection(metadata_list, body):
    
    #Preleva le informazioni nella lista metadata_list nel seguente ordine
    #FIELDS = ['sender', 'receiver', 'date', 'subject']
    #e le formatta in questa forma:
    #sender: xxxx | receiver: yyyy | date: wwww | subject: kkkk | body: zzzz
    
    metadata_prompt = (
        "sender: " + str(metadata_list[0]) + " | " +
        "receiver: " + str(metadata_list[1]) + " | " +
        "date: " + str(metadata_list[2]) + " | " +
        "subject: " + str(metadata_list[3])
    )

    body_text = metadata_prompt + " | body: " + body

    return body_text

In [ ]:
df_output_1 = pd.DataFrame()

for idx, row in df1.iterrows():

    #definizione della lista con le informazioni aggiuntive
    metadata_list = [
        row["sender"],
        row["receiver"],
        row["date"],
        row["subject"]
    ]
    if pd.isna(row["subject"]):
        metadata_list[3]=""             #Se il subject è NaN si fornisce un carattere vuoto.                                 

    body = row["body"]

    if pd.isna(body):
        body=""                         #body: NaN -> Body: ""

    prompt = compatta_detection(metadata_list, body)

    #Scrittura nel Dataframe di destinazione nel formato di Detection
    df_output_1.at[idx, "instruction"] = "You are a classification model specializing in emails, and your job is to detect phishing: respond only with \"0\" if it is not phishing or \"1\" if it is phishing, without explanations, symbols, additional letters, or other characters."
    df_output_1.at[idx, "text"] = prompt
    df_output_1.at[idx, "label"] = row["label"]


In [ ]:
len(df_output_1)        #Verifica che df_output_1 sia equivalente alla dimensione di df1

In [ ]:
len(df1)                # len(df_output_1) -> 25812  | len(df1) -> 25812 

In [ ]:
df_detection_label_1 = df_output_1.reset_index(drop=True)

df_detection_label_1.to_json("df_to_check_1.json", orient="records", indent=2)      #Scrittura in json

In [ ]:
dataset_metrics(df_detection_label_1)

## Pulizia dei testi ripetuti

Analizzando il dataset, molte mail risultano essere ripetuti nel corpo. Presentano le stesse frasi ripetute più volte senza un reale motivo, tramite questo codice successivo siamo in grado di filtrarle.

In [ ]:
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Dimensioni usate per definire gli N-Gram
NGRAM_SIZES = [7, 10, 15, 20]

#ECompressione di tanti spazi consecutivi in un unico
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

#Il corpo si trova successivamente a "body:", quindi viene estratto a partire dal compo text.
def extract_body(input_field):
    match = re.search(r'body:\s*(.*)', input_field, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""

#Costruzione a partire da un testo di diversi N-Gram
def build_ngrams(text, n):
    words = text.split(' ')
    return [
        ' '.join(words[i:i+n])
        for i in range(0, len(words), n)
        if len(words[i:i+n]) == n
    ]

#Funzione per trovare la migliore coppia
def best_pair_for_ngrams(ngrams):
    if len(ngrams) < 2:
        return None, None, -1.0

    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(ngrams)

    best_sim = -1.0
    best_i, best_j = None, None

    #Itera fra le diverse coppie e salva la migliore coppia e migliore similitudine
    for i in range(len(ngrams)):
        for j in range(i + 1, len(ngrams)):

            sim = cosine_similarity(tfidf[i], tfidf[j])[0][0]

            if sim > best_sim:
                best_sim = sim
                best_i, best_j = i, j

    return ngrams[best_i], ngrams[best_j], best_sim     #ritorno della coppia migliore e 
                                                        #della migliore similitudine


# ======================= MAIN =======================

json_path = "df_to_check_1.json"    #json di input
output_path = "df_1_checked.json"   #percorso di uscita
threshold = 0.6                     #se la similarità fra n-gram nel testo è maggiore di 0.6, il testo viene scartato

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

filtered_data = []         # <-- JSON finale senza duplicati
removed_count = 0
seen_count = 0

for idx, instance in enumerate(data, 1):
    seen_count += 1

    raw_input = instance.get("text", "")        
    
    body_text = extract_body(raw_input)     #estrae il corpo dal testo

    if not body_text:
        filtered_data.append(instance)      
        continue

    cleaned = clean_text(body_text)         #Pulizia testo da spazi in eccesso

    global_best_sim = -1.0
    global_best_pair = (None, None)
    global_best_n = None

    for n in NGRAM_SIZES:
        ngrams = build_ngrams(cleaned, n)
        ng1, ng2, sim = best_pair_for_ngrams(ngrams)
        if sim > global_best_sim:           #si verifica che la similitudine migliore
            global_best_sim = sim           #ottenuta è migliore di quella globale per
            global_best_pair = (ng1, ng2)   #il testo valutato
            global_best_n = n

    # === DECISIONE DI RIMOZIONE ===
    if global_best_sim > threshold:
        removed_count += 1
        removed = True
    else:
        removed = False
        filtered_data.append(instance)  # <-- ISTANZA TENUTA

    # Preview prime 5
    if idx <= 5:
        print(f"\n=== ISTANZA {idx} PREVIEW ===")
        print(f"Dimensione n-grammi: {global_best_n}")
        print("\nN-gramma 1:")
        print(global_best_pair[0])
        print("\nN-gramma 2:")
        print(global_best_pair[1])
        print(f"\nSimilarità (coseno TF-IDF): {global_best_sim:.4f}")

    # Stampa metriche (prime 5 + ogni 500)
    if idx <= 5 or seen_count % 500 == 0:
        print(
            f"[{seen_count}] removed={removed} | "
            f"best_sim={global_best_sim:.4f} | "
            f"removed_total={removed_count}/{seen_count}"
        )

# ===== SALVATAGGIO JSON FILTRATO =====
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print("\n=== RISULTATO FINALE ===")
print(f"Istanze originali: {len(data)}")
print(f"Istanze rimosse: {removed_count}")
print(f"Istanze finali: {len(filtered_data)}")
print(f"Percentuale rimossa: {(removed_count / seen_count) * 100:.2f}%")
print(f"JSON filtrato salvato in: {output_path}")

Caricamento dei risultati in df1_pulito

In [ ]:
import json
import pandas as pd


with open("df_1_checked.json", "r", encoding="utf-8") as f:
    data = json.load(f)             
    df1_pulito = pd.DataFrame(data)

In [ ]:
len(df1_pulito)     

In [ ]:
dataset_metrics(df1_pulito)

DF0_formattato   è il dataset df0 con l'aggiunte nel testo delle informazioni aggiuntive

DATASET_FINALE = DF con LABEL =0 + DF con LABEL = 1 pulito

In [ ]:
####Funzione ripetuta####
def compatta_detection(metadata_list, body):
    
    #Preleva le informazioni nella lista metadata_list nel seguente ordine
    #FIELDS = ['sender', 'receiver', 'date', 'subject']
    #e le formatta in questa forma:
    #sender: xxxx | receiver: yyyy | date: wwww | subject: kkkk | body: zzzz
    
    metadata_prompt = (
        "sender: " + str(metadata_list[0]) + " | " +
        "receiver: " + str(metadata_list[1]) + " | " +
        "date: " + str(metadata_list[2]) + " | " +
        "subject: " + str(metadata_list[3])
    )

    body_text = metadata_prompt + " | body: " + body

    return body_text

In [ ]:
#Formattazione di df0 in df0_formattato

df0_formattato = pd.DataFrame()

for idx, row in df0.iterrows():
    if idx % 3000 == 0:
        print(f"Processati {idx} record...")

    metadata_list = [
        row["sender"],
        row["receiver"],
        row["date"],
        row["subject"]
    ]
    if pd.isna(row["subject"]):
        metadata_list[3]=""

    body = row["body"]

    if pd.isna(body):
        body=""

    prompt = compatta_detection(metadata_list, body)

        # assegno i valori al DataFrame
    df0_formattato.at[idx, "instruction"] = "You are a classification model specializing in emails, and your job is to detect phishing: respond only with \"0\" if it is not phishing or \"1\" if it is phishing, without explanations, symbols, additional letters, or other characters."
    df0_formattato.at[idx, "text"] = prompt
    df0_formattato.at[idx, "label"] = row["label"]


In [ ]:
dataset_metrics(df0)
dataset_metrics(df0_formattato)

In [ ]:
#Pur di avere un Dataset fianle bilanciato, sono state mantenute solo le prime 16812 righe
#di df1_pulito. Grazie alle stampe precedenti notiamo che esistono 16812 istanze
#con label 0, quindi ne aggiungiamo altrettante con label 1

df1_pulito=df1_pulito.sample(frac=1).reset_index(drop=True)
prima_parte = df1_pulito.iloc[:16812].copy()
seconda_parte = df1_pulito.iloc[16812:].copy()
df_finale = pd.concat([df0_formattato, prima_parte], ignore_index=True)

In [ ]:
#df0_formattato.iloc[3]["text"]
#prima_parte.iloc[3]["text"]
#df0_formattato.iloc[3]["text"]
df_finale.iloc[3]["text"]           #stampa per vedere la corretta formattazione di df_final

In [ ]:
#Analisi Dataset

dataset_metrics(df_finale)

dataset_metrics(df0)
dataset_metrics(prima_parte)
dataset_metrics(seconda_parte)
dataset_metrics(df1_pulito)

In [ ]:
dataset_metrics(df_finale)

## Creazione Dataset Detection

In [ ]:
istogramma(df_detection,15,col_name="text")   #Con l'istogramma si possono notare degli outlier
                                              #con lunghezza eccessiva, questi vengono filtrati
                                              #tramite questo codice.

                                              #La tokenizzazione di prima ha fatto in modo da 
                                              #contare dei testi con token eccessivi, ma nel
                                              #testo erano ancora presenti dei testi con molti
                                              #spazi che incrementavano il numero di caratteri.

In [ ]:
print(len(df_finale))       #33624

df_finale = elimina_lunghe(df_finale, 4169, colonna="text")

print(len(df_finale))       #33619

Salvataggio del dataset di detection

In [ ]:
df_finale.to_json("dataset_detection_totale.json", orient="records", force_ascii=False, indent=4)

## Preparazione Dataset di Generazione

Rimozione "Original Message" dal corpo email, infatti le mail generate non dovrebbero rispondere a nessuna mail

In [ ]:
def remove_rows_by_pattern(df, pattern="Original Message", text_col="text"):


    if text_col not in df.columns:
        print(f"⚠️ Colonna '{text_col}' non trovata nel DataFrame")
        return df

    initial_rows = len(df)

    #Trova le entry con "Original Message nel corpo"
    mask = df[text_col].astype(str).str.contains(
        pattern,
        case=False,
        na=False
    )       

    df_clean = df[~mask].reset_index(drop=True)     #eliminzione
    removed = initial_rows - len(df_clean)

    print(f"Righe rimosse: {removed} (pattern: '{pattern}')")

    return df_clean

In [ ]:
dataset_metrics(df_finale)
count_pattern(df_finale)
df_senza_original_message = remove_rows_by_pattern(df_finale)
print(len(df_finale))
print(len(df_senza_original_message))

In [ ]:
dataset_metrics(df_senza_original_message)

In [ ]:
#Le mail che vogliamo generare sono le mail con label 1 in quanto vogliamo generare solo mail di phishing
df_generazione =df_senza_original_message[df_senza_original_message["label"]==1].reset_index(drop=True)
dataset_metrics(df_generazione)

In [ ]:
df_generazione.head()

In [ ]:
#Nelle mail di generazione non dobbiamo generare anche informazioni come "sender"
#"receiver" "date" 
df_output = df_generazione.copy()
df_output["text"] = df_output["text"].str.extract(
    r"\|\s*(subject:\s*.*)",
    flags=re.DOTALL
)


la generazione avviene tramite un formato diverso

"instruction":

"input":

"output":

In [ ]:
df_output = df_output.rename(columns={
    "text": "input",
    "label": "output"
}).reset_index(drop=True)

In [ ]:
df_output = df_output.iloc[:15000]  #accorcio la dimensione in quanto 15000 campioni sono
                                    #sufficienti per il nostro task

In [ ]:
dataset_metrics(df_output,label_col="output")
print(df_output.loc[1, "input"])

In [ ]:
#Creazione Dataset
df_output.to_json("df_generation_totale.json", orient="records", indent=2)

In [ ]:
df_output = pd.read_json("df_generation_totale.json")

In [ ]:
dataset_metrics(df_output)

## Split del Dataset in 3 parti

Per la fase di generazione è necessario generare un esempio di prompt a partire dalle mail di riferimento. Per questo motivo è stata usata una rete LLM, LLama3.1 70b, per generare i prompt a partire dal corpo del messaggio.

I dati così ottenuti:

* Prompt generato

* Email associata

Sono dati in ingresso a una rete di generazione di mail, per la fase di finetuning.

A causa delle risorse limitate della piattaforma è stato deciso per uno split in 3, in modo da generare i prompt singolarmente sui 3 dataset splittati.

In [ ]:
chunk_size = 5000

for i in range(3):
    start = i * chunk_size
    end = start + chunk_size
    
    df_chunk = df_output.iloc[start:end]
    
    filename = f"dataset_generation_parziale_{i+1}.json"
    df_chunk.to_json(
        filename,
        orient="records",
        force_ascii=False,
        indent=2
    )

    print(f"Creato {filename} con {len(df_chunk)} righe")

In [ ]:
df1 = pd.read_json("dataset_generation_parziale_1.json")
df2 = pd.read_json("dataset_generation_parziale_2.json")
df3 = pd.read_json("dataset_generation_parziale_3.json")

dataset_metrics(df1)
dataset_metrics(df2)
dataset_metrics(df3)

## Controllo Euristiche del Dataset di Detection

In [ ]:
df_detection = pd.read_json("dataset_detection_totale.json")

In [ ]:
import pandas as pd

df_generation = pd.read_json("df_generation_totale.json")
print(df_generation.columns.tolist())
df_generation = df_generation.rename(columns={
    'istruction': 'instruction',
}).reset_index(drop=True)
print(df_generation.columns.tolist())


## Controllo Euristiche del Dataset di Training della fase di Detection 

In [ ]:
df_detection = pd.read_json("train_dataset.json")

In [ ]:
dataset_metrics(df_detection)

analizza_dataset(df_detection,col_name="text")
print("Numero di file: ",conta_file(df_detection,col_name="text"))
count_pattern(df_detection,body_col="text")
count_keywords(df_detection,body_col="text",keywords=phishing_urgency_keywords)
istogramma(df_detection,15,col_name="text")

## Controllo Euristiche del Dataset di Testing della fase di Detection 

In [ ]:
df_detection = pd.read_json("test_dataset.json")

In [ ]:
dataset_metrics(df_detection)

analizza_dataset(df_detection,col_name="text")
print("Numero di file: ",conta_file(df_detection,col_name="text"))
count_pattern(df_detection,body_col="text")
count_keywords(df_detection,body_col="text",keywords=phishing_urgency_keywords)
istogramma(df_detection,15,col_name="text")

## Controllo Euristiche del Dataset di Generation 

In [ ]:
df_generation = pd.read_json("df_generation_totale.json")

In [ ]:
dataset_metrics(df_generation,label_col="output")

analizza_dataset(df_generation,col_name="input")
print("Numero di file: ",conta_file(df_generation,col_name="input"))
count_pattern(df_generation,body_col="input")
count_keywords(df_generation,body_col="input",keywords=phishing_urgency_keywords)
istogramma(df_generation,15,col_name="input")